In [1]:
import time
import pickle
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
# fix professors
from owlready2 import *
main_onto_name = 'main-ResearchMentorOntology.owl'
onto_path.append("/Users/lauramiron/Desktop/CS270/project")
onto_path.append("http://www.w3.org/2003/06/sw-vocab-status/ns#")
onto_path.append("http://www.w3.org/2002/07/owl#")
onto_path.append("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
onto_path.append("http://www.w3.org/XML/1998/namespace")
onto_path.append("http://www.w3.org/2001/XMLSchema")
onto_path.append("http://vivoweb.org/ontology/core")
onto_path.append("http://www.w3.org/2000/01/rdf-schema")
onto_path.append("http://purl.org/dc/terms")
onto = get_ontology(main_onto_name)
vivoNS = onto.get_namespace("http://vivoweb.org/ontology/core")
onto.load()

get_ontology("http://www.semanticweb.org/lauramiron/ontologies/2019/1/merged-research-mentor#")

In [23]:
# scrape biox students

biox_student_url = 'https://biox.stanford.edu/about/people/profiles-by-year?title&field_group_tid=29&field_award_year_value%5Bvalue%5D%5Byear%5D=&page='
driver = webdriver.Chrome()
bio_student_dict = {}
print('scraping biox graduate fellows...')
for pg_num in range(0,27):
    print(pg_num)
    url = biox_student_url+str(pg_num)
    driver.get(url)
    time.sleep(3)
    profiles = driver.find_elements_by_class_name("node-profile")
    for prof in profiles:
        try:
            name = prof.find_element_by_tag_name('a').get_attribute('innerText').split('-')[0].strip()
            first_name = name.split(' ')[0]
            last_name = name.split(' ')[1]
            indiv_name = first_name+last_name
            info_node = prof.find_element_by_class_name('field-type-text-with-summary')
            awarded_text = info_node.find_element_by_class_name('field-item').get_attribute('innerText')
            year = int(awarded_text.split('\n')[0].split(' ')[2])
            print(year)
            if year >= 2014:
                print(first_name, last_name)
                data = {'firstName':first_name,'lastName':last_name, 'faculty':[]}
#                 facs = re.split('(and)|(,)',awarded_text.split('\n')[2].split(':')[1].strip())
                facs = re.sub('(and)|(\(.*\))','000',awarded_text.split('\n')[2].split(':')[1].strip())
                for f in facs.split('000'):
                    if f==None: continue
                    fac = f.strip().strip('and').strip().strip(',').strip()
                    iname = ''
                    for nm in fac.split(' '):
                        print(nm)
                        iname += nm
                    data['faculty'].append(iname)
                    print(iname)
                bio_student_dict[indiv_name] = data
        except Exception as e:
            continue
driver.quit()
with open('biox_phd.p','wb+') as f:
    pickle.dump(bio_student_dict,f)


In [3]:
# scrape med students
med_students_base_url = 'https://med.stanford.edu/profiles/browse?p={0}&affiliations=capMdStudent&ps=100'

driver = webdriver.Chrome()
med_student_dict = {}
print('scraping med students...')
for pg_num in range(1,4):
    print('downloading page '+str(pg_num))
    url = med_students_base_url.format(pg_num)
    driver.get(url)
    time.sleep(8)
    profiles = driver.find_elements_by_class_name("mini-profile")
    for pfl in profiles:
        media_heading_elem = pfl.find_element_by_class_name("media-heading")
        url_name = media_heading_elem.get_attribute("href").split('/')[-1]
        name = media_heading_elem.find_element_by_tag_name('h4').get_attribute('innerText')
        first_name = name.split(' ')[0].replace(' ','')
        last_name = name.split(' ')[-1].replace(' ','')
        program_info = media_heading_elem.find_element_by_tag_name('h5').get_attribute('innerText')
        program = program_info.split(' ')[0]
        grad_year = program_info.split(' ')[-1]
        med_student_dict[url_name] = {'firstName':first_name, 'lastName': last_name, 'program': program, 'gradDate': grad_year}
driver.quit()
med_student_dict_name = 'med_student_simple.p'
print('saving to '+med_student_dict_name)        
with open(med_student_dict_name,'wb+') as f:
    pickle.dump(med_student_dict,f)

NameError: name 'webdriver' is not defined

In [7]:
# add med students to ontology
# med_student_profile_url = 'https://med.stanford.edu/profiles/'
# main_onto_name = 'main-ResearchMentorOntology.owl'
# med_student_dict_name = 'med_student_simple.p'
# med_student_dict = pickle.load(open(med_student_dict_name,'rb'))

# for url_name, values in med_student_dict.items():
#     first_name = values['firstName']
#     last_name = values['lastName']
#     program = values['program']
#     grad_year = values['gradDate']
    
#     med_indiv = onto.MedicalStudent((first_name+last_name).replace(' ',''))
#     med_indiv.firstName = first_name
#     med_indiv.lastName = last_name
#     med_indiv.graduationDateExp.append(grad_year)
#     med_indiv.uri = url_name
#     med_indiv.hasProfilePage.append(med_student_profile_url+url_name)

# onto.save()

biox_student_profile_url = 'https://biox.stanford.edu/about/people/undergraduate-fellows/{0}-bio-x-sigf-fellow'
biox_student_dict_name = 'biox_phd.p'
biox_student_dict = pickle.load(open(biox_student_dict_name,'rb'))

for iname, values in biox_student_dict.items():
    if ('(' in iname) or ('"' in iname): continue
    first_name = values['firstName']
    last_name = values['lastName']
#     print(iname)
    indiv = onto.DoctoralStudent(iname)
    indiv.firstName = first_name
    indiv.lastName = last_name
    profile_url = biox_student_profile_url.format(first_name+'-'+last_name)
#     indiv.hasProfilePage.append(profile_url)
    for fac in values['faculty']:
        if not fac: continue
        print(fac)
        fac_indiv = onto.search_one(iri='*'+fac,is_a=vivoNS.FacultyMember)
        if (fac_indiv):
            print(fac_indiv)
#             print(type(fac_indiv))
#             print(fac_indiv.name)
            indiv.mentoredBy.append(fac_indiv)
onto.save()

SamSanjivGambhir


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
# data sources
# med school students
# https://med.stanford.edu/profiles/browse?p=1&affiliations=capMdStudent&ps=100

# biox undergraduates
#'https://biox.stanford.edu/about/people/profiles-by-year?title&field_group_tid=31&field_award_year_value%5Bvalue%5D%5Byear%5D=&page='

# biox graduates
#https://biox.stanford.edu/about/people/profiles-by-year?title&field_group_tid=29&field_award_year_value%5Bvalue%5D%5Byear%5D=